In [1]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler

realCIR = np.load('real.npy')
imagCIR = np.load('imag.npy')
magCIR = np.load('dataset.npy')
GT = np.load('label.npy')

print(realCIR.shape, imagCIR.shape, magCIR.shape, GT.shape)

tan = np.arctan2(imagCIR, realCIR)
diff = tan[:,:63] - tan[:, 1:]
diffrence = np.zeros((magCIR.shape[0],magCIR.shape[1]))
diffrence[:,1:] = diff

array = np.stack([magCIR,diffrence],axis=-1)
# array = np.stack([magCIR,tan],axis=-1)

X_train, X_test, y_train, y_test = train_test_split(array, GT, test_size=0.2, shuffle=True, stratify=GT)

(18000, 64) (18000, 64) (18000, 64) (18000,)


In [2]:
model = tf.keras.models.Sequential([
    # layer1~3
    tf.keras.layers.Conv1D(64,3, activation=tf.nn.relu,input_shape=(64,2),padding='same'),
    tf.keras.layers.Conv1D(64,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.AveragePooling1D(2,padding='same'),
    # layer4~6
    tf.keras.layers.Conv1D(128,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Conv1D(128,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.AveragePooling1D(2,padding='same'),
    # layer7~11
    tf.keras.layers.Conv1D(256,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Conv1D(256,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Conv1D(256,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Conv1D(256,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.AveragePooling1D(2,padding='same'),
    # layer12~16
    tf.keras.layers.Conv1D(512,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Conv1D(512,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Conv1D(512,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Conv1D(512,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.AveragePooling1D(2,padding='same'),
    # layer17~21
    tf.keras.layers.Conv1D(512,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Conv1D(512,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Conv1D(512,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Conv1D(512,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.AveragePooling1D(2,padding='same'),
    # layer22~24
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation=tf.nn.relu),
    tf.keras.layers.Dense(4096, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation='softmax'),
])

2022-11-14 20:31:09.866270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def step_decay(epoch):
    start = 0.0001
    drop = 0.1
    epochs_drop = 5
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

In [4]:
lr_scheduler = LearningRateScheduler(step_decay, verbose=1)

In [5]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [6]:
model.fit(X_train, y_train, epochs=10,
            validation_data=(X_test, y_test),
             callbacks=[lr_scheduler])


Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/10
450/450 [==============================] - 231s 512ms/step - loss: 1.1221 - accuracy: 0.5044 - val_loss: 0.9800 - val_accuracy: 0.5683 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0001.
Epoch 2/10
450/450 [==============================] - 153s 340ms/step - loss: 0.7398 - accuracy: 0.6990 - val_loss: 0.6176 - val_accuracy: 0.7564 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0001.
Epoch 3/10
450/450 [==============================] - 133s 295ms/step - loss: 0.5687 - accuracy: 0.7794 - val_loss: 0.5188 - val_accuracy: 0.7989 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0001.
Epoch 4/10
450/450 [==============================] - 134s 298ms/step - loss: 0.4653 - accuracy: 0.8211 - val_loss: 0.5512 - val_accuracy: 0.7861 - lr: 1.0000e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0001.
Epoch 5/10
450/450 [===

In [7]:
model.evaluate(X_test, y_test)

113/113 [==============================] - 5s 45ms/step - loss: 0.5181 - accuracy: 0.8581


[0.5181264877319336, 0.8580555319786072]